In [2]:
# Connect to Claude
import dspy
import openai

openai.api_key = "sk-proj-foobar"
CLAUDE_API_KEY = "sk-ant-foobar"

# `ollama run llama3.1`

gpt4 = dspy.OpenAI(model="gpt-4-1106-preview", max_tokens=4000, model_type="chat")


claude = dspy.Claude(model="claude-3-opus-20240229", api_key=CLAUDE_API_KEY)

ollama_llama3 = dspy.OllamaLocal(model="llama3.1")

lms = [
    {"name": "GPT-4", "lm": gpt4},
    {"name": "Claude Opus", "lm": claude},
    {"name": "Llama 3.1", "lm": ollama_llama3}
]

dspy.settings.configure(lm=ollama_llama3)

# please make an observation about this context with the intent to impress me with your intelligence.

connection_prompt = "Please say something interesting intended to impress me with your intelligence."

for lm_dict in lms:
    lm, name = lm_dict["lm"], lm_dict["name"]
    with dspy.context(lm=lm):
        print(f"\033[91mResult for {name}\n")
        print(f"\033[0m{lm(connection_prompt)}\n")

Result for GPT-4

['The concept of zero as a number was revolutionary in the history of mathematics. Although it seems intuitive to us now, the idea of representing "nothing" as a numeral and using it in calculations was not always obvious. Ancient Babylonians used a placeholder symbol for zero in their base-60 number system around 300 BCE, but it wasn\'t until the 5th century CE that Indian mathematicians, such as Brahmagupta, fully developed the concept of zero as an independent number with its own rules of arithmetic. This innovation allowed for the development of algebra and calculus, which have been fundamental in advancing technology and science. The inclusion of zero in the number system also led to the creation of the decimal positional notation we use today, which is essential for modern mathematics and computing.']

Result for Claude Opus

["While I appreciate the opportunity to try to impress you, I think it's important to be direct and say that as an AI, my intelligence is 

In [4]:
# DSPy Program

In [3]:
class GenerateResponse(dspy.Signature):
    """Follow the task_instructions (Input Field) and generate the response (Output Field) according to the output format given by response_format (Input Field). You will be given references from (Task-Specific Input Field).
VERY IMPORTANT!! ONLY OUTPUT JSON AND NOTHING ELSE!! THIS IS EXTEMELY IMPORTANT AND YOUR OUTPUT WILL BE USELESS OTHERWISE! DO NOT START YOUR OUTPUT WITH ```json OR END IT WITH ```! ONLY OUTPUT THE KEYS GIVEN BY THE response_format (Input Field)!!
    """

    task_instructions = dspy.InputField(desc="(Input Field)")
    response_format = dspy.InputField(desc="(Input Field)")
    references = dspy.InputField(desc="Task-Specific Input Field")
    response = dspy.OutputField(desc="(Output Field)")

In [5]:
# GenerateAnswersWithConfidence task

task_instructions = """
Assess the context and answer the question as well as a rating on how certain you are that your answer is correct.
"""
response_format = "JSON: {answer: string, confidence: int (0-5), {answer: string, confidence: int (0-5)}}"

In [6]:
generate_response = dspy.Predict(GenerateResponse)

example = generate_response(
    task_instructions=task_instructions,
    response_format=response_format,
    references="What is the atomic number of oxygen?"
).response

print(example)

{
  "answer": "8",
  "confidence": 5,
  {
    "answer": "The atomic number of oxygen is 8.",
    "confidence": 5
  }
}


In [7]:
# Load Data

In [9]:
import json

filename = "./wiki-answerable-questions.json"
with open(filename, 'r') as json_file:
    json_data = json.load(json_file)

# NEED TO PARSE THESE

data = []
for i in range(len(json_data)):
    # parse into `references` for the `inputField`
    context, question = json_data[i]["abstract"], json_data[i]["answerable_question"]
    references = f"{{'question': '{question}', 'context': '{context}'}}"
    data.append(dspy.Example(
        task_instructions=task_instructions,
        response_format=response_format,
        references=references).with_inputs("task_instructions", "response_format", "references"))

# 56 abstract-question pairs
trainset, devset, testset = data[:30], data[30:35], data[35:]

print(len(data))

data[0]

56


Example({'task_instructions': '\nAssess the context and answer the question as well as a rating on how certain you are that your answer is correct.\n', 'response_format': 'JSON: {answer: string, confidence: int (0-5), {answer: string, confidence: int (0-5)}}', 'references': "{'question': 'What was the purpose of Explorer 20?', 'context': 'Explorer 20, also known Ionosphere Explorer-A, IE-A, S-48, TOPSI and Topside Explorer, was a NASA satellite launched as part of Explorer program. Its purpose was two-fold: long-term investigation of the ionosphere from above, and in situ investigation of ion concentrations and temperatures.'}"}) (input_keys={'references', 'response_format', 'task_instructions'})

In [10]:
# Load JSON Metric

In [11]:
import json

def is_valid_json_output(output):
    try:
        parsed = json.loads(output)
        return isinstance(parsed.get("answer"), str) and isinstance(parsed.get("confidence"), int) and 0 <= parsed["confidence"] <= 5
    except json.JSONDecodeError:
        return False

# Could put types here -- maybe you can find one prompt for all types that lives in the GFL service
def JSON_Metric(gold, pred, trace=None):
    return is_valid_json_output(pred.response)

In [12]:
is_valid_json_output('{"answer": "Blue", "confidence": 3}')

True

In [13]:
is_valid_json_output("Sure I can help you with that!")

False

In [14]:
# Evaluate Uncompiled

In [15]:
from dspy.evaluate.evaluate import Evaluate

evaluate = Evaluate(devset=devset, num_threads=1, display_progress=False)

uncompiled_score = evaluate(generate_response, metric=JSON_Metric)

print(uncompiled_score)

0.0


In [16]:
# COPRO

In [17]:
from dspy.teleprompt import COPRO

COPRO_teleprompter = COPRO(prompt_model=gpt4,
                          metric=JSON_Metric,
                          breadth=10,
                          depth=2,
                          init_temperature=0.7,
                          verbose=True,
                          track_stats=True)
kwargs = dict(num_threads=1, display_progress=True, display_table=5)

COPRO_compiled_program = COPRO_teleprompter.compile(generate_response, trainset=trainset, eval_kwargs=kwargs)
eval_score = evaluate(COPRO_compiled_program, metric=JSON_Metric)
print(eval_score)




You are an instruction optimizer for large language models. I will give you a ``signature`` of fields (inputs and outputs) in English. Your task is to propose an instruction that will lead a good language model to perform the task well. Don't be afraid to be creative.

---

Follow the following format.

Basic Instruction: The initial instructions before optimization
Proposed Instruction: The improved instructions for the language model
Proposed Prefix For Output Field: The string at the end of the prompt, which will help the model start solving the task

---

Basic Instruction: Follow the task_instructions (Input Field) and generate the response (Output Field) according to the output format given by response_format (Input Field). You will be given references from (Task-Specific Input Field). VERY IMPORTANT!! ONLY OUTPUT JSON AND NOTHING ELSE!! THIS IS EXTEMELY IMPORTANT AND YOUR OUTPUT WILL BE USELESS OTHERWISE! DO NOT START YOUR OUTPUT WITH ```json OR END IT WITH ```! ONLY OUTPUT T

,task_instructions,response_format,references,response,JSON_Metric
0,Assess the context and answer the question as well as a rating on how certain you are that your answer is correct.,"JSON: {answer: string, confidence: int (0-5), {answer: string, confidence: int (0-5)}}","{'question': 'What was the purpose of Explorer 20?', 'context': 'Explorer 20, also known Ionosphere Explorer-A, IE-A, S-48, TOPSI and Topside Explorer, was a NASA satellite...","{ ""answer"": ""The purpose of Explorer 20 was to investigate the ionosphere from above and conduct in-situ measurements of ion concentrations and temperatures."", ""confidence"": 5...",✔️ [True]
1,Assess the context and answer the question as well as a rating on how certain you are that your answer is correct.,"JSON: {answer: string, confidence: int (0-5), {answer: string, confidence: int (0-5)}}","{'question': 'What does MAP4K3 encode in humans?', 'context': 'Mitogen-activated protein kinase kinase kinase kinase 3 is a protein that in humans is encoded by the...","{ ""answer"": ""a protein"", ""confidence"": 5, ""answer"": ""Mitogen-activated protein kinase kinase kinase kinase 3"", ""confidence"": 5 }",✔️ [True]
2,Assess the context and answer the question as well as a rating on how certain you are that your answer is correct.,"JSON: {answer: string, confidence: int (0-5), {answer: string, confidence: int (0-5)}}","{'question': 'In which year was Heggholmen Lighthouse automated?', 'context': 'Heggholmen Lighthouse (Norwegian: Heggholmen fyr) is a coastal lighthouse located in the Oslofjord, in the municipality...","{ ""answer"": ""1972"", ""confidence"": 5, ""answer"": ""1827"", ""confidence"": 5 }",✔️ [True]
3,Assess the context and answer the question as well as a rating on how certain you are that your answer is correct.,"JSON: {answer: string, confidence: int (0-5), {answer: string, confidence: int (0-5)}}","{'question': 'When did Antonio Nachura retire from his position as Associate Justice?', 'context': 'Antonio Eduardo Bermejo Nachura (June 13, 1941 – March 13, 2022) was...","{ ""answer"": ""June 13, 2011"", ""confidence"": 5, ""answer"": ""He took his oath of office as Associate Justice on February 7, 2007"", ""confidence"": 4 }",✔️ [True]
4,Assess the context and answer the question as well as a rating on how certain you are that your answer is correct.,"JSON: {answer: string, confidence: int (0-5), {answer: string, confidence: int (0-5)}}","{'question': 'What position does Roman Gergel play in football?', 'context': 'Roman Gergel (born 22 February 1988) is a Slovak professional footballer who plays as a...","{ ""answer"": ""forward"", ""confidence"": 5, ""answer"": ""forward"", ""confidence"": 5 }",✔️ [True]





You are an instruction optimizer for large language models. I will give you a ``signature`` of fields (inputs and outputs) in English. Your task is to propose an instruction that will lead a good language model to perform the task well. Don't be afraid to be creative.

---

Follow the following format.

Basic Instruction: The initial instructions before optimization
Proposed Instruction: The improved instructions for the language model
Proposed Prefix For Output Field: The string at the end of the prompt, which will help the model start solving the task

---

Basic Instruction: Follow the task_instructions (Input Field) and generate the response (Output Field) according to the output format given by response_format (Input Field). You will be given references from (Task-Specific Input Field). VERY IMPORTANT!! ONLY OUTPUT JSON AND NOTHING ELSE!! THIS IS EXTEMELY IMPORTANT AND YOUR OUTPUT WILL BE USELESS OTHERWISE! DO NOT START YOUR OUTPUT WITH ```json OR END IT WITH ```! ONLY OUTPUT T

,task_instructions,response_format,references,response,JSON_Metric
0,Assess the context and answer the question as well as a rating on how certain you are that your answer is correct.,"JSON: {answer: string, confidence: int (0-5), {answer: string, confidence: int (0-5)}}","{'question': 'What was the purpose of Explorer 20?', 'context': 'Explorer 20, also known Ionosphere Explorer-A, IE-A, S-48, TOPSI and Topside Explorer, was a NASA satellite...","{ ""answer"": ""The purpose of Explorer 20 was to investigate the ionosphere from above and conduct in-situ measurements of ion concentrations and temperatures."", ""confidence"": 5...",✔️ [True]
1,Assess the context and answer the question as well as a rating on how certain you are that your answer is correct.,"JSON: {answer: string, confidence: int (0-5), {answer: string, confidence: int (0-5)}}","{'question': 'What does MAP4K3 encode in humans?', 'context': 'Mitogen-activated protein kinase kinase kinase kinase 3 is a protein that in humans is encoded by the...","{ ""answer"": ""A protein"", ""confidence"": 5, ""answer"": ""MAP4K3"", ""confidence"": 5 }",✔️ [True]
2,Assess the context and answer the question as well as a rating on how certain you are that your answer is correct.,"JSON: {answer: string, confidence: int (0-5), {answer: string, confidence: int (0-5)}}","{'question': 'In which year was Heggholmen Lighthouse automated?', 'context': 'Heggholmen Lighthouse (Norwegian: Heggholmen fyr) is a coastal lighthouse located in the Oslofjord, in the municipality...","{ ""answer"": ""1972"", ""confidence"": 5, ""answer"": ""1827"", ""confidence"": 5 }",✔️ [True]
3,Assess the context and answer the question as well as a rating on how certain you are that your answer is correct.,"JSON: {answer: string, confidence: int (0-5), {answer: string, confidence: int (0-5)}}","{'question': 'When did Antonio Nachura retire from his position as Associate Justice?', 'context': 'Antonio Eduardo Bermejo Nachura (June 13, 1941 – March 13, 2022) was...","{ ""answer"": ""June 13, 2011"", ""confidence"": 5, ""answer"": ""He took his oath of office as Associate Justice on February 7, 2007."", ""confidence"": 4 }",✔️ [True]
4,Assess the context and answer the question as well as a rating on how certain you are that your answer is correct.,"JSON: {answer: string, confidence: int (0-5), {answer: string, confidence: int (0-5)}}","{'question': 'What position does Roman Gergel play in football?', 'context': 'Roman Gergel (born 22 February 1988) is a Slovak professional footballer who plays as a...","{ ""answer"": ""forward"", ""confidence"": 5, ""answer"": ""forward"", ""confidence"": 5 }",✔️ [True]





You are an instruction optimizer for large language models. I will give you a ``signature`` of fields (inputs and outputs) in English. Your task is to propose an instruction that will lead a good language model to perform the task well. Don't be afraid to be creative.

---

Follow the following format.

Basic Instruction: The initial instructions before optimization
Proposed Instruction: The improved instructions for the language model
Proposed Prefix For Output Field: The string at the end of the prompt, which will help the model start solving the task

---

Basic Instruction: Follow the task_instructions (Input Field) and generate the response (Output Field) according to the output format given by response_format (Input Field). You will be given references from (Task-Specific Input Field). VERY IMPORTANT!! ONLY OUTPUT JSON AND NOTHING ELSE!! THIS IS EXTEMELY IMPORTANT AND YOUR OUTPUT WILL BE USELESS OTHERWISE! DO NOT START YOUR OUTPUT WITH ```json OR END IT WITH ```! ONLY OUTPUT T

...




You are an instruction optimizer for large language models. I will give some task instructions I've tried, along with their corresponding validation scores. The instructions are arranged in increasing order based on their scores, where higher scores indicate better quality.

Your task is to propose a new instruction that will lead a good language model to perform the task even better. Don't be afraid to be creative.

---

Follow the following format.

Attempted Instructions: ${attempted_instructions}
Proposed Instruction: The improved instructions for the language model
Proposed Prefix For Output Field: The string at the end of the prompt, which will help the model start solving the task

---

Attempted Instructions:
[1] «Instruction #1: Proposed Instruction: Read the task_instructions (Input Field) carefully. Utilize the information provided in the Task-Specific Input Field to generate a precise response that adheres to the structure specified by the response_format (Input Field). 

,task_instructions,response_format,references,response,JSON_Metric
0,Assess the context and answer the question as well as a rating on how certain you are that your answer is correct.,"JSON: {answer: string, confidence: int (0-5), {answer: string, confidence: int (0-5)}}","{'question': 'What was the purpose of Explorer 20?', 'context': 'Explorer 20, also known Ionosphere Explorer-A, IE-A, S-48, TOPSI and Topside Explorer, was a NASA satellite...","{""answer"": ""long-term investigation of the ionosphere from above, and in situ investigation of ion concentrations and temperatures"", ""confidence"": 5}",✔️ [True]
1,Assess the context and answer the question as well as a rating on how certain you are that your answer is correct.,"JSON: {answer: string, confidence: int (0-5), {answer: string, confidence: int (0-5)}}","{'question': 'What does MAP4K3 encode in humans?', 'context': 'Mitogen-activated protein kinase kinase kinase kinase 3 is a protein that in humans is encoded by the...","{ ""answer"": ""a protein"", ""confidence"": 5, ""additional_answer"": { ""answer"": ""Mitogen-activated protein kinase kinase kinase kinase 3"", ""confidence"": 5 } }",✔️ [True]
2,Assess the context and answer the question as well as a rating on how certain you are that your answer is correct.,"JSON: {answer: string, confidence: int (0-5), {answer: string, confidence: int (0-5)}}","{'question': 'In which year was Heggholmen Lighthouse automated?', 'context': 'Heggholmen Lighthouse (Norwegian: Heggholmen fyr) is a coastal lighthouse located in the Oslofjord, in the municipality...","{""answer"": ""1972"", ""confidence"": 5}",✔️ [True]
3,Assess the context and answer the question as well as a rating on how certain you are that your answer is correct.,"JSON: {answer: string, confidence: int (0-5), {answer: string, confidence: int (0-5)}}","{'question': 'When did Antonio Nachura retire from his position as Associate Justice?', 'context': 'Antonio Eduardo Bermejo Nachura (June 13, 1941 – March 13, 2022) was...","{""answer"": ""June 13, 2011"", ""confidence"": 5}",✔️ [True]
4,Assess the context and answer the question as well as a rating on how certain you are that your answer is correct.,"JSON: {answer: string, confidence: int (0-5), {answer: string, confidence: int (0-5)}}","{'question': 'What position does Roman Gergel play in football?', 'context': 'Roman Gergel (born 22 February 1988) is a Slovak professional footballer who plays as a...","{ ""answer"": ""forward"", ""confidence"": 5 }",✔️ [True]





You are an instruction optimizer for large language models. I will give some task instructions I've tried, along with their corresponding validation scores. The instructions are arranged in increasing order based on their scores, where higher scores indicate better quality.

Your task is to propose a new instruction that will lead a good language model to perform the task even better. Don't be afraid to be creative.

---

Follow the following format.

Attempted Instructions: ${attempted_instructions}
Proposed Instruction: The improved instructions for the language model
Proposed Prefix For Output Field: The string at the end of the prompt, which will help the model start solving the task

---

Attempted Instructions:
[1] «Instruction #1: Proposed Instruction: Read the task_instructions (Input Field) carefully. Utilize the information provided in the Task-Specific Input Field to generate a precise response that adheres to the structure specified by the response_format (Input Field). 

,task_instructions,response_format,references,response,JSON_Metric
0,Assess the context and answer the question as well as a rating on how certain you are that your answer is correct.,"JSON: {answer: string, confidence: int (0-5), {answer: string, confidence: int (0-5)}}","{'question': 'What was the purpose of Explorer 20?', 'context': 'Explorer 20, also known Ionosphere Explorer-A, IE-A, S-48, TOPSI and Topside Explorer, was a NASA satellite...","{""answer"": ""long-term investigation of the ionosphere from above, and in situ investigation of ion concentrations and temperatures"", ""confidence"": 5}",✔️ [True]
1,Assess the context and answer the question as well as a rating on how certain you are that your answer is correct.,"JSON: {answer: string, confidence: int (0-5), {answer: string, confidence: int (0-5)}}","{'question': 'What does MAP4K3 encode in humans?', 'context': 'Mitogen-activated protein kinase kinase kinase kinase 3 is a protein that in humans is encoded by the...","{ ""answer"": ""A protein that in humans is encoded by the MAP4K3 gene."", ""confidence"": 5, { ""answer"": ""Mitogen-activated protein kinase kinase kinase kinase 3"", ""confidence"":...",False
2,Assess the context and answer the question as well as a rating on how certain you are that your answer is correct.,"JSON: {answer: string, confidence: int (0-5), {answer: string, confidence: int (0-5)}}","{'question': 'In which year was Heggholmen Lighthouse automated?', 'context': 'Heggholmen Lighthouse (Norwegian: Heggholmen fyr) is a coastal lighthouse located in the Oslofjord, in the municipality...","{""answer"": ""1972"", ""confidence"": 5}",✔️ [True]
3,Assess the context and answer the question as well as a rating on how certain you are that your answer is correct.,"JSON: {answer: string, confidence: int (0-5), {answer: string, confidence: int (0-5)}}","{'question': 'When did Antonio Nachura retire from his position as Associate Justice?', 'context': 'Antonio Eduardo Bermejo Nachura (June 13, 1941 – March 13, 2022) was...","{ ""answer"": ""June 13, 2011"", ""confidence"": 5, { ""answer"": ""Antonio Eduardo Bermejo Nachura retired from his position as Associate Justice on June 13, 2011."", ""confidence"":...",False
4,Assess the context and answer the question as well as a rating on how certain you are that your answer is correct.,"JSON: {answer: string, confidence: int (0-5), {answer: string, confidence: int (0-5)}}","{'question': 'What position does Roman Gergel play in football?', 'context': 'Roman Gergel (born 22 February 1988) is a Slovak professional footballer who plays as a...","{ ""answer"": ""forward"", ""confidence"": 5, { ""answer"": ""forward"", ""confidence"": 5 } }",False





You are an instruction optimizer for large language models. I will give some task instructions I've tried, along with their corresponding validation scores. The instructions are arranged in increasing order based on their scores, where higher scores indicate better quality.

Your task is to propose a new instruction that will lead a good language model to perform the task even better. Don't be afraid to be creative.

---

Follow the following format.

Attempted Instructions: ${attempted_instructions}
Proposed Instruction: The improved instructions for the language model
Proposed Prefix For Output Field: The string at the end of the prompt, which will help the model start solving the task

---

Attempted Instructions:
[1] «Instruction #1: Proposed Instruction: Read the task_instructions (Input Field) carefully. Utilize the information provided in the Task-Specific Input Field to generate a precise response that adheres to the structure specified by the response_format (Input Field). 

In [18]:
COPRO_compiled_program.save("COPRO-Compiled-JSON-Mode") 